In [1]:
import pandas as pd
import os
import re
from Bio import SeqIO
import subprocess 
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import seaborn as sns

%matplotlib inline

In [79]:
os.chdir('/nasdata/Neev/pacbio_data/VL/')
pacbioData='/nasdata/Neev/pacbio_data/VL/m54158_190405_195049.subreads.fastq'
vectorFa='/nasdata/Neev/pacbio_data/VL/ecoli_pWEB_TNC2000bp.fasta'
#find vector in sequences
cmd="minimap2 -a "+vectorFa+" "+pacbioData+" | samtools sort -@ 60 -o ALL.sorted.bam -"
print cmd
subprocess.call(cmd, shell= True) 
subprocess.call(["samtools view -@ 60 -b -F 4 ALL.sorted.bam > coli_vector_mapped.bam"], shell= True)
subprocess.call(["samtools view -b -f 4 ALL.sorted.bam > unmapped.bam"], shell= True)
#contigs without vector are kept as is
subprocess.call(["samtools bam2fq unmapped.bam > unmapped.fastq"], shell= True)

#contigs with vector have their non-aligned (clipped) parts extracted
subprocess.call(["java -jar /home/vlibis/jvarkit/dist/samextractclip.jar --minsize 1000 -o /nasdata/Neev/pacbio_data/VL/soft_clipped.fq /nasdata/Neev/pacbio_data/VL/coli_vector_mapped.bam"], shell= True)

#merge contigs without vector and non-vector clipped parts from the other contigs
subprocess.call(["cat soft_clipped.fq unmapped.fastq > 2249_all_raw_reads_clipped_vector2000bp.fq"], shell= True)


minimap2 -a /nasdata/Neev/pacbio_data/VL/ecoli_pWEB_TNC2000bp.fasta /nasdata/Neev/pacbio_data/VL/m54158_190405_195049.subreads.fastq | samtools sort -@ 60 -o ALL.sorted.bam -


0

In [ ]:
#run FLYE assembler
cmd="flye --pacbio-raw /nasdata/Neev/pacbio_data/VL/2249_all_raw_reads_clipped_vector2000bp.fq --out-dir FLYE_2249_noColi_Vector2000bp --genome-size 200m --threads 60"
subprocess.call(cmd, shell= True) 

## clean large contigs from vector:

In [80]:
#find T7 M13 and split contigs 
M13=['TCCCAGTCACGACGGATCCC','TCGCGGCCGCCAGGGTTT',
       'AAACCCTGGCGGCCGCGA','GGGATCCGTCGTGACTGGGA']
T7= ['GATCCTCTCCCTATAGTGAG','TCGTATTATGCGGCCGCGA',
      'TCGCGGCCGCATAATACGA','CTCACTATAGGGAGAGGATC']
with open('/nasdata/Neev/pacbio_data/VL/FLYE_2249_noColi_Vector2000bp/scaffolds.fasta', "rU") as input_handle:
    with open('/nasdata/Neev/pacbio_data/VL/FLYE_2249_noColi_Vector2000bp/valid_scaffolds_edgesRemoved.fasta', "w") as output_handle:
        counter=0
        counterBothEdges=0
        sequences = SeqIO.parse(input_handle, "fasta") 
        for record in sequences:
  
            if len(record.seq)>25000:
                fragments=[str(record.seq)]
                M13flag=False
                T7flag=False
                
                for motif in M13:             
                    for frag in fragments:
                        offsprings=re.split(motif, frag)
                        if len(offsprings)==2:            
                            M13flag=True
                            fragments.extend(offsprings)
                            fragments.remove(frag)
                for motif in T7:             
                    for frag in fragments:
                        offsprings=re.split(motif, frag)
                        if len(offsprings)==2:            
                            T7flag=True
                            fragments.extend(offsprings)
                            fragments.remove(frag)
                
                    
                for idx,frag in enumerate(fragments):
                    if len(frag)>25000 and len(frag)<55000 and T7flag and M13flag:
                        counter+=1
                        fragID=record.id+'_'+str(idx)
                        output_handle.write(">%s\n%s\n" % (fragID,frag))
                
print str(counter) + ' valid contigs with both T7 and M13'


2314 valid contigs with both T7 and M13


In [83]:
#double-check for leftover vector 
pacbioData='/nasdata/Neev/pacbio_data/VL/FLYE_2249_noColi_Vector2000bp/valid_scaffolds_edgesRemoved.fasta'
vectorFa='/nasdata/Neev/pacbio_data/VL/pWEB_TNC_linear_noM13T7.fasta'
cmd="minimap2 -a "+vectorFa+" "+pacbioData+" | samtools sort -@ 60 -o ALL.sorted.bam -"
print cmd
subprocess.call(cmd, shell= True) 
subprocess.call(["samtools view -b -f 4 ALL.sorted.bam > unmapped.bam"], shell= True)
subprocess.call(["samtools fasta unmapped.bam > 2249_valid_scaffolds_NOvector.fasta"], shell= True)

minimap2 -a /nasdata/Neev/pacbio_data/VL/pWEB_TNC_linear_noM13T7.fasta /nasdata/Neev/pacbio_data/VL/FLYE_2249_noColi_Vector2000bp/valid_scaffolds_edgesRemoved.fasta | samtools sort -@ 60 -o ALL.sorted.bam -


0